# Quick Start With Public Datasets and add new Dataset

## Introduction

In this tutorial, we introduce how to quickly set up workflows with MONAI public Datasets and how to add new Dataset.  
Currently, MONAI provides `MedNISTDataset` and `DecathlonDataset` to automatically download,  
extract MedNIST dataset and Decathlon dataset, and act as PyTorch datasets to generate training/validation/test data. 

We'll cover the following topics in this tutorial:
<ul>
    <li>Create training experiment with MedNISTDataset and workflow</li>
    <li>Create training experiment with DecathlonDataset and workflow</li>
    <li>Share other public data and add Dataset in MONAI</li>
</ul>


In [ ]:
import torch
import shutil
import tempfile
import logging
import os
import sys
from typing import Any, Callable, Optional
import matplotlib.pyplot as plt
from ignite.metrics import Accuracy
from monai.data import DataLoader, CacheDataset
from monai.apps import MedNISTDataset, DecathlonDataset, download_and_extract
from monai.transforms import \
    LoadPNGd, AddChanneld, ScaleIntensityd, ToTensord, Compose, Randomizable, \
    AsDiscreted, LoadNiftid, Spacingd, Orientationd, Resized, AsDiscreted
from monai.networks.nets import densenet121, UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss
from monai.engines import SupervisedTrainer
from monai.inferers import SimpleInferer
from monai.handlers import StatsHandler, MeanDice

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Create training experiment with MedNISTDataset and workflow
The MedNIST dataset was gathered from several sets from [TCIA](https://wiki.cancerimagingarchive.net/display/Public/Data+Usage+Policies+and+Restrictions), [the RSNA Bone Age Challenge](http://rsnachallenges.cloudapp.net/competitions/4), and [the NIH Chest X-ray dataset](https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest).

### Set up pre-processing transforms

In [ ]:
transform = Compose(
    [
        LoadPNGd(keys='image'),
        AddChanneld(keys='image'),
        ScaleIntensityd(keys='image'),
        ToTensord(keys='image')
    ]
)

### Create MedNISTDataset for training
`MedNISTDataset` inherits from MONAI `CacheDataset` and provides rich parameters to achieve expected behavior:
1. **root_dir**: target directory to download and load MedNIST dataset.
2. **section**: expected data section, can be: `training`, `validation` or `test`.
3. **transform**: transforms to execute operations on input data. the default transform is composed by `LoadPNGd` and `AddChanneld`, which can load data into numpy array with [C, H, W] shape.
4. **download**: whether to download and extract the MedNIST from resource link, default is False. if expected file already exists, skip downloading even set it to True. user can manually copy `MedNIST.tar.gz` file or `MedNIST` folder to root directory.
5. **seed**: random seed to randomly split training, validation and test datasets, defaut is 0.
6. **val_frac**: percentage of of validation fraction in the whole dataset, default is 0.1.
7. **test_frac**: percentage of of test fraction in the whole dataset, default is 0.1.
8. **cache_num**: number of items to be cached. Default is `sys.maxsize`. will take the minimum of (cache_num, data_length x cache_rate, data_length).
9. **cache_rate**: percentage of cached data in total, default is 1.0 (cache all). will take the minimum of (cache_num, data_length x cache_rate, data_length).
10. **num_workers**: the number of worker threads to use. If 0 a single thread will be used. Default is 0.

Note that the "tar files" are cached after a first time downloading. the `self.__getitem__()` API generates 1 `{"image": XXX, "label": XXX}` dict according to the specified index within the `self.__len__()`.

In [ ]:
tempdir = tempfile.mkdtemp()
train_ds = MedNISTDataset(root_dir=tempdir, transform=transform, section="training", download=True)
# the dataset can work seamlessly with the pytorch native dataset loader,
# but using monai.data.DataLoader has additional benefits of mutli-process
# random seeds handling, and the customized collate functions
train_loader = DataLoader(train_ds, batch_size=300, shuffle=True, num_workers=10)

### Pick images from MedNISTDataset to visualize and check

In [ ]:
plt.subplots(3, 3, figsize=(8, 8))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(train_ds[i * 5000]['image'][0].detach().cpu(), cmap='gray')
plt.tight_layout()
plt.show()

### Create training components

In [ ]:
device = torch.device('cuda:0')
net = densenet121(spatial_dims=2, in_channels=1, out_channels=6).to(device)
loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(net.parameters(), 1e-5)

### Define the easiest training workflow and run
Use MONAI SupervisedTrainer handlers to quickly set up a training workflow.

In [ ]:
trainer = SupervisedTrainer(
    device=device,
    max_epochs=5,
    train_data_loader=train_loader,
    network=net,
    optimizer=opt,
    loss_function=loss,
    inferer=SimpleInferer(),
    key_train_metric={'train_acc': Accuracy(output_transform=lambda x: (x['pred'], x['label']))},
    train_handlers=StatsHandler(tag_name='train_loss', output_transform=lambda x: x['loss']),
)
trainer.run()

shutil.rmtree(tempdir)

## Create training experiment with DecathlonDataset and workflow
The Decathlon dataset came from [Medical Segmentation Decathlon](http://medicaldecathlon.com/) AI challenge.

### Set up pre-processing transforms

In [ ]:
transform = Compose(
    [
        LoadNiftid(keys=['image', 'label']),
        AddChanneld(keys=['image', 'label']),
        Spacingd(keys=['image', 'label'], pixdim=(1., 1., 1.), mode=('bilinear', 'nearest')),
        Orientationd(keys=['image', 'label'], axcodes='RAS'),
        ScaleIntensityd(keys='image'),
        Resized(keys=['image', 'label'], spatial_size=(32, 64, 32), mode=('trilinear', 'nearest')),
        ToTensord(keys=['image', 'label']),
    ]
)

### Create DeccathlonDataset for training
`DecathlonDataset` inherits from MONAI `CacheDataset` and provides rich parameters to achieve expected behavior:
1. **root_dir**: user's local directory for caching and loading the MSD datasets.
2. **task**: which task to download and execute: one of list ("Task01_BrainTumour", "Task02_Heart", "Task03_Liver", "Task04_Hippocampus", "Task05_Prostate", "Task06_Lung", "Task07_Pancreas", "Task08_HepaticVessel", "Task09_Spleen", "Task10_Colon").
3. **section**: expected data section, can be: `training`, `validation` or `test`.
4. **transform**: transforms to execute operations on input data. the default transform is composed by `LoadNiftid` and `AddChanneld`, which can load data into numpy array with [C, H, W, D] shape.
5. **download**: whether to download and extract the Decathlon from resource link, default is False. if expected file already exists, skip downloading even set it to True. user can manually copy tar file or dataset folder to the root directory.
6. **seed**: random seed to randomly split `training`, `validation` and `test` datasets, defaut is 0.
7. **val_frac**: percentage of of validation fraction from the `training` section, default is 0.2. Decathlon data only contains `training` section with labels and `test` section without labels, so randomly select fraction from the `training` section as the `validation` section.
8. **cache_num**: number of items to be cached. Default is `sys.maxsize`. will take the minimum of (cache_num, data_length x cache_rate, data_length).
9. **cache_rate**: percentage of cached data in total, default is 1.0 (cache all). will take the minimum of (cache_num, data_length x cache_rate, data_length).
10. **num_workers**: the number of worker threads to use. if 0 a single thread will be used. Default is 0.

Note that the "tar files" are cached after a first time downloading. the `self.__getitem__()` API generates 1 `{"image": XXX, "label": XXX}` dict according to the specified index within the `self.__len__()`.

In [ ]:
tempdir = tempfile.mkdtemp()
train_ds = DecathlonDataset(root_dir=tempdir, task='Task04_Hippocampus', transform=transform,
                            section='training', download=True)
# the dataset can work seamlessly with the pytorch native dataset loader,
# but using monai.data.DataLoader has additional benefits of mutli-process
# random seeds handling, and the customized collate functions
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=16)

### Pick images from DecathlonDataset to visualize and check

In [ ]:
plt.subplots(3, 3, figsize=(8, 8))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(train_ds[i * 20]['image'][0, :, :, 10].detach().cpu(), cmap='gray')
plt.tight_layout()
plt.show()

### Create training components

In [ ]:
device = torch.device('cuda:0')
net = UNet(dimensions=3, in_channels=1, out_channels=3, channels=(16, 32, 64, 128, 256),
           strides=(2, 2, 2, 2), num_res_units=2, norm=Norm.BATCH).to(device)
loss = DiceLoss(to_onehot_y=True, softmax=True)
opt = torch.optim.Adam(net.parameters(), 1e-2)

### Define the easiest training workflow and run
Use MONAI SupervisedTrainer handlers to quickly set up a training workflow.

In [ ]:
trainer = SupervisedTrainer(
    device=device,
    max_epochs=5,
    train_data_loader=train_loader,
    network=net,
    optimizer=opt,
    loss_function=loss,
    inferer=SimpleInferer(),
    post_transform=AsDiscreted(keys=['pred', 'label'], argmax=(True, False), to_onehot=True, n_classes=3),
    key_train_metric={'train_meandice': MeanDice(output_transform=lambda x: (x['pred'], x['label']))},
    train_handlers=StatsHandler(tag_name='train_loss', output_transform=lambda x: x['loss']),
)
trainer.run()

shutil.rmtree(tempdir)

## Share other public data and add Dataset in MONAI
Referring to the `MedNISTDataset` or `DecathlonDataset`, it's easy to create a new Dataset fo other public data.  
Mainly include below steps:
1. Inherit MONAI `CacheDataset` to leverage caching mechanism to accelerate training.
1. Make sure the lisence of dataset allows public access and share.
2. Use `monai.apps.download_and_extract` to download and extract data in the Dataset.
3. Define the logic to randomly split `training`, `validation` and `test` sections.
4. Construct data list with `dict` items:
```py
[
    {'image': image1_path, 'label': label1_path},
    {'image': image2_path, 'label': label2_path},
    {'image': image3_path, 'label': label3_path},
    ... ...
]
```
5. Define dataset specific logic.

### Define IXIDataset as an example
Here we use the [IXI Dataset](https://brain-development.org/ixi-dataset/) as an example to show how to create a new `IXIDataset`.

In [ ]:
class IXIDataset(Randomizable, CacheDataset):
    resource = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T1.tar'
    md5 = '34901a0593b41dd19c1a1f746eac2d58'

    def __init__(
        self,
        root_dir: str,
        section: str,
        transform: Callable[..., Any],
        download: bool = False,
        seed: int = 0,
        val_frac: float = 0.2,
        test_frac: float = 0.2,
        cache_num: int = sys.maxsize,
        cache_rate: float = 1.0,
        num_workers: int = 0,
    ):
        if not os.path.isdir(root_dir):
            raise ValueError('root_dir must be a directory.')
        self.section = section
        self.val_frac = val_frac
        self.test_frac = test_frac
        self.set_random_state(seed=seed)
        dataset_dir = os.path.join(root_dir, 'ixi')
        tarfile_name = f"{dataset_dir}.tar"
        if download:
            download_and_extract(self.resource, tarfile_name, dataset_dir, self.md5)
        # as a quick demo, we just use 10 images to show
        self.datalist = [
            {'image': os.path.join(dataset_dir, 'IXI314-IOP-0889-T1.nii.gz'), 'label': 0},
            {'image': os.path.join(dataset_dir, 'IXI249-Guys-1072-T1.nii.gz'), 'label': 0},
            {'image': os.path.join(dataset_dir, 'IXI609-HH-2600-T1.nii.gz'), 'label': 0},
            {'image': os.path.join(dataset_dir, 'IXI173-HH-1590-T1.nii.gz'), 'label': 1},
            {'image': os.path.join(dataset_dir, 'IXI020-Guys-0700-T1.nii.gz'), 'label': 0},
            {'image': os.path.join(dataset_dir, 'IXI342-Guys-0909-T1.nii.gz'), 'label': 0},
            {'image': os.path.join(dataset_dir, 'IXI134-Guys-0780-T1.nii.gz'), 'label': 0},
            {'image': os.path.join(dataset_dir, 'IXI577-HH-2661-T1.nii.gz'), 'label': 1},
            {'image': os.path.join(dataset_dir, 'IXI066-Guys-0731-T1.nii.gz'), 'label': 1},
            {'image': os.path.join(dataset_dir, 'IXI130-HH-1528-T1.nii.gz'), 'label': 0}
        ]
        data = self._generate_data_list()
        super().__init__(data, transform, cache_num=cache_num, cache_rate=cache_rate, num_workers=num_workers)


    def randomize(self, data: Optional[Any] = None):
        self.rann = self.R.random()

    def _generate_data_list(self):
        data = list()
        for d in self.datalist:
            self.randomize()
            if self.section == 'training':
                if self.rann < self.val_frac + self.test_frac:
                    continue
            elif self.section == 'validation':
                if self.rann >= self.val_frac:
                    continue
            elif self.section == 'test':
                if self.rann < self.val_frac or self.rann >= self.val_frac + self.test_frac:
                    continue
            else:
                raise ValueError('section name can only be: training, validation or test.')
            data.append(d)
        return data

### Pick images from IXIDataset to visualize and check

In [ ]:
tempdir = tempfile.mkdtemp()
train_ds = IXIDataset(
    root_dir=tempdir,
    section='training',
    transform=Compose([LoadNiftid('image'), ToTensord('image')]),
    download=True
)
plt.figure('check', (18, 6))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    plt.imshow(train_ds[i]['image'][:, :, 80].detach().cpu(), cmap='gray')
plt.show()

shutil.rmtree(tempdir)